In [1]:
# How to Clean Up Your Data
## Example: Where to look for Meteorites in the USA

So you have some fresh data, which you are sure contains useful insights. But now what?

While every dataset will need its own specific cleaning steps, we follow a general procedure, no matter the data we are cleaning. I will use as an example the Meteorite Landings dataset provided by the Meteorical Society, accessible at the NASA Open Data Portal (this webpage) [https://data.nasa.gov/Space-Science/Meteorite-Landings/ak9y-cwf9]. I will data clean using Python 3.9 in Microsoft Visual Studio Code. 

In any data cleaning exercise, we follow these steps:

### Step 1: Define your initial goal.

Carefully define what you want from the data. You may revise it, but at first, think about what you want to accomplish. For the Meteorite Landings data, we want to find where we can most readily find meteorites in the continental USA. Assuming that meteorite landings are spread evenly across the surface of the Earth, this would correspond to the area with the *least* density of meteorites which have been found.

### Step 2: Describe your Dataset and Summarize its attributes.

This basic step involves doing very basic perusals of your data. How many data entries does the data contain? How many data fields? What does each data field contain? 

Pandas is our favorite library for answering these basic questions, because of its ease of use and many handy methods for data exploration. If you haven't already, create a virtual environment in your project directory, activate it, and then install pandas.

* python 3.9 -m venv .venv *(Create a Virtual Environment in your current directory)*
* .venv\Scripts\Activate.ps1 *(activate it in your PowerShell terminal)*
* pip install pandas *(install pandas!)*

Now open your data and inspect it using. I'll do this for Meteorite Landings as an example. This data is in CSV, handy to read using pandas -- pandas will read the CSV and convert into a tabular dataframe. I will inspect the summary information of this dataframe and then inspect a sample using two handy dataframe methods - sample(), and the atttribute .style.

In [2]:
import pandas as pd

meteorite_data = pd.read_csv('example-datasets/Meteorite_Landings.csv')

meteorite_data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'example-datasets/Meteorite_Landings.csv'

The dataset is 4.2 MB in size, with 11 fields composed of a mix of strings and float values. Let's look at a random sample of 5 data points.

In [3]:
meteorite_data.sample(5).style

NameError: name 'meteorite_data' is not defined